In [ ]:
%%capture
!pip install super-gradients
!pip install --upgrade pillow
!pip install --upgrade torchvision
!pip install super-gradients
!pip install torch
!pip install ultralytics

In [ ]:
import os
import random
import torch
import requests
from PIL import Image
from ultralytics import YOLO, NAS
import cv2
import requests
import json
from ultralytics import models
from super_gradients.training import Trainer, dataloaders, models
from super_gradients.training.losses import PPYoloELoss
from super_gradients.training.metrics import DetectionMetrics_050

[2024-05-30 06:55:10] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it


The console stream is logged into /root/sg_logs/console.log


[2024-05-30 06:55:17] WARNING - env_sanity_check.py - Failed to verify installed packages: boto3 required but not found
[2024-05-30 06:55:17] WARNING - env_sanity_check.py - Failed to verify installed packages: deprecated required but not found
[2024-05-30 06:55:17] WARNING - env_sanity_check.py - Failed to verify installed packages: coverage required but not found
[2024-05-30 06:55:17] WARNING - env_sanity_check.py - Failed to verify installed packages: sphinx-rtd-theme required but not found
[2024-05-30 06:55:17] WARNING - env_sanity_check.py - Failed to verify installed packages: torchmetrics required but not found
[2024-05-30 06:55:17] WARNING - env_sanity_check.py - Failed to verify installed packages: hydra-core required but not found
[2024-05-30 06:55:17] WARNING - env_sanity_check.py - Failed to verify installed packages: omegaconf required but not found
[2024-05-30 06:55:17] WARNING - env_sanity_check.py - Failed to verify installed packages: onnxruntime required but not found

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks

In [ ]:
car_detection_model_path = '/content/drive/MyDrive/Colab Notebooks/Models/CarDetection.pt'
parking_spot_detection_model_path = '/content/drive/MyDrive/Models/Colab Notebooks/SpotsDetection.pt'
car_detection_4_plates_model_path = '/content/drive/MyDrive/Colab Notebooks/Models/bestcar.pt'
plate_detection_model_path = '/content/drive/MyDrive/Colab Notebooks/Models/bestplate.pt'

In [ ]:
car_detection_model = YOLO(car_detection_model_path)
car_detection_4_plate_model = YOLO(car_detection_4_plates_model_path)
plate_detection_model = YOLO(plate_detection_model_path)


In [ ]:
parking_spot_detection_model = models.get(model_name = 'yolo_nas_l', pretrained_weights = 'coco')


[2024-05-30 06:57:00] WARNING - checkpoint_utils.py - :warning: The pre-trained models provided by SuperGradients may have their own licenses or terms and conditions derived from the dataset used for pre-training.
 It is your responsibility to determine whether you have permission to use the models for your use case.
 The model you have requested was pre-trained on the coco dataset, published under the following terms: https://cocodataset.org/#termsofuse
[2024-05-30 06:57:00] INFO - checkpoint_utils.py - License Notification: YOLO-NAS pre-trained weights are subjected to the specific license terms and conditions detailed in 
https://github.com/Deci-AI/super-gradients/blob/master/LICENSE.YOLONAS.md
By downloading the pre-trained weight files you agree to comply with these terms.
Downloading: "https://sghub.deci.ai/models/yolo_nas_l_coco.pth" to /root/.cache/torch/hub/checkpoints/yolo_nas_l_coco.pth
100%|██████████| 256M/256M [00:17<00:00, 15.5MB/s]
[2024-05-30 06:57:20] INFO - checkpoin

In [ ]:
checkpoint_path = "/content/drive/MyDrive/Colab Notebooks/checkpoints/race_number/RUN_20240523_220735_495739/ckpt_best.pth"

if not os.path.exists(checkpoint_path):
    print(f"Checkpoint file not found: {checkpoint_path}")
else:
    print(f"Checkpoint file found: {checkpoint_path}")

    parking_spot_detection_model = models.get(model_name = 'yolo_nas_l',
                            num_classes=1,
                            checkpoint_path=checkpoint_path)

Checkpoint file found: /content/drive/MyDrive/Colab Notebooks/checkpoints/race_number/RUN_20240523_220735_495739/ckpt_best.pth


[2024-05-30 06:58:23] INFO - checkpoint_utils.py - Successfully loaded model weights from /content/drive/MyDrive/Colab Notebooks/checkpoints/race_number/RUN_20240523_220735_495739/ckpt_best.pth EMA checkpoint.


In [ ]:
from PIL import Image
from ultralytics import YOLO
import cv2

import numpy as np
from PIL import Image
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator, Colors

def plot_bounding_boxes(yolo_result,image_choosen, colors=Colors()):
    """Draw bounding boxes from yolo.predict() result"""
    # image_choosen = '1'
    image = yolo_result.orig_img[..., ::-1]
    annotate = Annotator(np.ascontiguousarray(image))
    classes = yolo_result.boxes.cls.tolist()
    scores = yolo_result.boxes.conf.tolist()
    boxes = yolo_result.boxes.xyxy
    image = cv2.imread(image_path)
    width, height = image.shape[1], image.shape[0]



    for box, class_, score in zip(boxes, classes, scores):
        color = np.random.randint(0, 255, 3).tolist()
        tag = f"{yolo_result.names[class_].title()}: {score:.0%}"
        annotate.box_label(box, tag, color)



    return Image.fromarray(annotate.result())

In [ ]:
def convert_to_chars(num_list):
    char_map = {
        '00': '0',
        '01': '1',
        '05': '2',
        '07': '3',
        '03': '4',
        '02': '5',
        '06': '6',
        '04': '7',
        '08': '8',
        '10': '9',
        '22': 'ا',
        '11': 'ب',
        '24': 'ح',
        '28': 'د',
        '20': 'ر',
        '12': 'س',
        '25': 'ص',
        '21': 'ط',
        '26': 'ع',
        '10': 'ق',
        '17': 'ك',
        '18': 'ل',
        '13': 'م',
        '19': 'ن',
        '14': 'ه',
        '23': 'و',
        '16': 'ي'
    }

    converted_chars = []
    for num_str in num_list:
        if num_str in char_map:
            converted_chars.append(char_map[num_str])
        else:
            converted_chars.append('X')

    # Sort the converted_chars list
    converted_chars.sort()
    return converted_chars

In [ ]:
async def platesProcess(image):
    dh, dw, _ = image.shape
    conf = 0.3

    result = car_detection_4_plate_model(image, conf=conf, verbose=False)[0]
    highest_conf_index = 0
    highest_conf = 0

    bounding_boxes = plot_bounding_boxes_car(result)
    if bounding_boxes:
        for i, box in enumerate(bounding_boxes):
            if len(box) == 5:
                class_, x, y, w, h = box
                if class_ > highest_conf:
                    highest_conf = class_
                    highest_conf_index = i
            else:
                continue

        try:
            x, y, w, h = bounding_boxes[highest_conf_index]
            x = int((x - w / 2) * dw)
            y = int((y - h / 2) * dh)
            w = int(w * dw)
            h = int(h * dh)

            imgCrop = image[y:y+h, x:x+w]
            plates_nums = PredictPlate(imgCrop)

            return image, convert_to_chars(plates_nums)
        except IndexError:
            print("Error: No valid bounding box found.")
            return image, []
    else:
        print("Error: No bounding boxes returned.")
        return image, []

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
from ultralytics.utils.plotting import Annotator, Colors
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def plot_bounding_boxes_plate(yolo_result):
    """Draw bounding boxes from yolo.predict() result"""
    classes = []
    if yolo_result.boxes:
        for class_ in yolo_result.boxes.cls.tolist():
            classes.append(yolo_result.names[class_])
    return classes

In [ ]:
def PredictPlate(image):
    conf= 0.3


    result = plate_detection_model(image, conf=conf, verbose=False)[0]
    platePredictions = plot_bounding_boxes_plate(result)

    return platePredictions

In [ ]:
def PredictParking(image):
    conf = 0.60
    predictions = parking_spot_detection_model.predict(image, conf=conf)
    bboxes = predictions.prediction.bboxes_xyxy
    count = len(bboxes)
    width, height = image.shape[1], image.shape[0]
    parking_bboxes = []
    for bbox in predictions.prediction.bboxes_xyxy:
        x1, y1, x2, y2 = bbox
        x1 = x1 / width
        y1 = y1 / height
        x2 = x2 / width
        y2 = y2 / height
        parking_bboxes.append([ x1, y1, x2, y2])

    return  parking_bboxes, count

In [ ]:
def plot_bounding_boxes_car(yolo_result, colors=Colors()):
    """Draw bounding boxes from yolo.predict() result"""
    boxes =[]
    for x, y, w, h in yolo_result.boxes.xywhn.tolist():
        boxes.append([x, y, w, h])

    return boxes


In [ ]:
def PredictCars(image):
    conf= 0.3
    car_detection_model = YOLO(car_detection_model_path)
    result = car_detection_model(image)[0]
    Carspredictions = plot_bounding_boxes_car(result)
    return Carspredictions, len(Carspredictions)

In [ ]:
video_path = '/content/drive/MyDrive/Colab Notebooks/Colab Notebooks/Data/boxes/Screenshot 2024-05-29 102057.png'
image = cv2.imread(video_path)

fr, pa = await platesProcess(image)
pa

In [ ]:
!pip install websockets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 3.2 MB/s eta 0:00:00


In [ ]:
!pip install pyngrok

In [ ]:
def draw_bounding_boxes(image, parking_bboxes, car_bboxes,iou_sum_matrix):
    # image_path = "/content/drive/MyDrive/Colab Notebooks/Colab Notebooks/Data/datasets/train/images/10.png"
    # image = cv2.imread(image_path)
    # x1, y1, x2, y2
    # count of free parking and count of incorrectly prak
    count_free_park =0
    count_incorrectly_prak = 0
    for i, park in enumerate(iou_sum_matrix):
        if park <= 0.05:
            count_free_park= count_free_park +1
            # print('The number of free parking is : ', count_free_park)
        elif park >0.05 and park <=0.5:
            count_incorrectly_prak = count_incorrectly_prak +1

    print('The number of free parking is : ', count_free_park)
    print('The number of incorrectly parking is : ', count_incorrectly_prak)
    fre_p = count_free_park
    incor_p = count_incorrectly_prak



    parkingbboxes = []
    for parking_count,pbox in enumerate(parking_bboxes,1):
        start_point = (int(pbox[0] * image.shape[1]), int(pbox[1] * image.shape[0]))
        end_point = (int(pbox[2] * image.shape[1]), int(pbox[3] * image.shape[0]))
        count_free_park =0
        # for i, park in enumerate(iou_sum_matrix):
        if iou_sum_matrix[parking_count-1] <= 0.05:
            cv2.rectangle(image, start_point, end_point, (0, 255, 0), 2)
            cv2.putText(image, f"Parking {parking_count}", (int(pbox[0] * image.shape[1]), int(pbox[1] * image.shape[0]-10)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            count_free_park= count_free_park +1
        elif iou_sum_matrix[parking_count-1] >0.05 and iou_sum_matrix[parking_count-1] <=0.5:
            cv2.rectangle(image, start_point, end_point, (0, 0, 255), 2)
            cv2.putText(image, f"Parking {parking_count}", (int(pbox[0] * image.shape[1]), int(pbox[1] * image.shape[0]-10)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        else:
            cv2.rectangle(image, start_point, end_point, (0, 165, 255), 2)
            cv2.putText(image, f"Parking {parking_count}", (int(pbox[0] * image.shape[1]), int(pbox[1] * image.shape[0]-10)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 165, 255), 2)


    carbboxes =[]
    for values in car_bboxes:
        xc = float(values[0])
        yc = float(values[1])
        w = float(values[2])
        h = float(values[3])
        carbboxes.append([xc, yc, w, h])

    for car_count, cbbox in enumerate(carbboxes, 1):
        xc = int(cbbox[0] * image.shape[1])
        yc = int(cbbox[1] * image.shape[0])
        w = int(cbbox[2] * image.shape[1])
        h = int(cbbox[3] * image.shape[0])
        cv2.rectangle(image, (xc - w // 2, yc - h // 2), (xc + w // 2, yc + h // 2), (255, 0, 0), 2)
        # cv2.putText(image, f"Car {car_count}", (xc - w // 2, yc - h // 2 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    img=image


    return img, fre_p, incor_p

In [ ]:
def calculate_iou(box1, box2):
    (x1, y1, x2, y2), (x, y, w, h) = box1[-4:], box2[-4:]
    # box1 = [x1 * image.shape[1], y1  * image.shape[0], x2 * image.shape[1], y2  * image.shape[0]]
    box2 = [x - w / 2, y - h / 2, w, h]

    # Calculate intersection coordinates
    x_intersection = max(box1[0], box2[0])
    y_intersection = max(box1[1], box2[1])
    #                        x_max ,  x_max =x+w
    x_overlap = max(0, min( box1[2], box2[0] + box2[2]) - x_intersection)
    #                      y_max  , y_max= y+h
    y_overlap = max(0, min(box1[3], box2[1] + box2[3]) - y_intersection)

    # Calculate areas
    intersection_area = x_overlap * y_overlap
    #           ( x_max - x_min  ) * (y_max - y_min)
    box1_area = ((box1[2] - box1[0]) * (box1[3] - box1[1]))
      #               w*h
    box2_area = box2[2] * box2[3]
    union_area = box1_area + box2_area - intersection_area

    # Calculate IoU
    iou = intersection_area / union_area
    return iou

# Function to calculate IoU between all pairs of bounding boxes
def calculate_iou_matrix(boxes1, boxes2):
    iou_matrix = []
    for box1 in boxes1:
        iou_row = []
        for box2 in boxes2:
            iou_row.append(calculate_iou(box1, box2))
        iou_matrix.append(iou_row)
    return iou_matrix
# def sum_matrix_iou(image):





In [ ]:
import concurrent.futures

async def makeAnnotation(frame):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        car_boxes_future = executor.submit(PredictCars, frame)
        parking_boxes_future = executor.submit(PredictParking, frame)

        car_boxes, car_count = car_boxes_future.result()
        parking_boxes, parking_count = parking_boxes_future.result()
        iou_matrix = calculate_iou_matrix(parking_boxes, car_boxes)

    # Print IoU matrix
    iou_sum_matrix =[]
    park_number = []
    for i, row in enumerate(iou_matrix):
        # print(f"IoU scores for box {i+1} in parking:")
        iou_sum = 0.0
        for j, iou_score in enumerate(row):
            # print(f"  With box {j+1} in car: {iou_score}")
            iou_sum= iou_sum + float(iou_score)
        iou_sum_matrix.append(iou_sum)
        park_number.append(i+1)
        # print(f"IoU sum {iou_sum_matrix[i]} in parking:")

    image_bb, free, wrong = draw_bounding_boxes(frame, parking_boxes, car_boxes,iou_sum_matrix)
#    Newframe = draw_bounding_boxes(frame, parking_boxes, car_boxes)
    return image_bb, car_count, parking_count, free, wrong


In [ ]:
port = 8548

In [ ]:
from pyngrok import ngrok, conf
port = port+1
conf.get_default().auth_token = 'USE THE AUTH TOKEN FROM NGROK' # IMPORTANT NOTE
print('Your app is running here:', ngrok.connect(port).public_url)

[2024-05-30 08:58:58] INFO - ngrok.py - Opening tunnel named: http-8558-27f2a528-4858-4a54-815a-be4ddcc105a4
[2024-05-30 08:58:58] INFO - process.py - Overriding default auth token
[2024-05-30 08:58:58] INFO - process.py - t=2024-05-30T08:58:58+0000 lvl=info msg="no configuration paths supplied"
[2024-05-30 08:58:58] INFO - process.py - t=2024-05-30T08:58:58+0000 lvl=info msg="using configuration at default config path" path=/root/.config/ngrok/ngrok.yml
[2024-05-30 08:58:58] INFO - process.py - t=2024-05-30T08:58:58+0000 lvl=info msg="open config file" path=/root/.config/ngrok/ngrok.yml err=nil
[2024-05-30 08:58:58] INFO - process.py - t=2024-05-30T08:58:58+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040 allow_hosts=[]
[2024-05-30 08:58:59] INFO - process.py - t=2024-05-30T08:58:59+0000 lvl=info msg="client session established" obj=tunnels.session
[2024-05-30 08:58:59] INFO - process.py - t=2024-05-30T08:58:59+0000 lvl=info msg="tunnel session started" obj=tunnels

In [ ]:
import asyncio
import base64
import json
import numpy as np
import cv2
import websockets
import nest_asyncio
from websockets.exceptions import ConnectionClosedError



async def process_video(websocket, path):
    print("Client connected")
    try:
        async for message in websocket:
            print("Frame received")
            message_data = json.loads(message)
            image_data = message_data.get('imageData')
            video_number = message_data.get('videoNumber')

            if not image_data:
                continue

            image_bytes = image_data.split('data:image/jpeg;base64,')[1]
            image_np = np.frombuffer(base64.b64decode(image_bytes), dtype=np.uint8)
            frame = cv2.imdecode(image_np, cv2.IMREAD_COLOR)

            if video_number == 1:
                frame, car_count,  parking_count, free, wrong = await makeAnnotation(frame)
                annotations = {
                    'car count': car_count,
                    'parking count': parking_count,
                    'free parking': free,
                    'wrong parking': wrong,
                    'frame': f'data:image/jpeg;base64,{base64.b64encode(cv2.imencode(".jpg", frame)[1]).decode("utf-8")}',
                    'videoNumber': video_number
                }
            elif video_number == 2:
                print("got video 2")
                frame, platesNums = await platesProcess(frame)
                annotations = {
                    'plates Nums': platesNums,
                    'frame': f'data:image/jpeg;base64,{base64.b64encode(cv2.imencode(".jpg", frame)[1]).decode("utf-8")}',
                    'videoNumber': video_number
                }

            await websocket.send(json.dumps(annotations))
            print("Frame processed and sent")
    except ConnectionClosedError:
        print("Client disconnected")
    except Exception as e:
        print(f"Error processing frame: {e}")

nest_asyncio.apply()
start_server = websockets.serve(process_video, '127.0.0.1', port)

print("Server started")
asyncio.get_event_loop().run_until_complete(start_server)
asyncio.get_event_loop().run_forever()

[2024-05-30 08:59:00] INFO - server.py - server listening on 127.0.0.1:8558


Server started


[2024-05-30 09:00:12] INFO - process.py - t=2024-05-30T09:00:12+0000 lvl=info msg="join connections" obj=join id=b7b8c519e2c3 l=127.0.0.1:8558 r=37.224.52.18:54030
[2024-05-30 09:00:12] INFO - server.py - connection rejected (426 Upgrade Required)
[2024-05-30 09:00:12] INFO - server.py - connection closed
[2024-05-30 09:00:12] INFO - process.py - t=2024-05-30T09:00:12+0000 lvl=info msg="join connections" obj=join id=bba36b3f69c3 l=127.0.0.1:8558 r=37.224.52.18:54030
[2024-05-30 09:00:12] INFO - server.py - connection rejected (426 Upgrade Required)
[2024-05-30 09:00:12] INFO - server.py - connection closed
[2024-05-30 09:01:12] INFO - process.py - t=2024-05-30T09:01:12+0000 lvl=info msg="join connections" obj=join id=a62687a91333 l=127.0.0.1:8558 r=37.224.52.18:60635
[2024-05-30 09:01:12] INFO - server.py - connection open


Client connected
Frame received
got video 2
Frame processed and sent
Frame received
got video 2
Frame processed and sent
Frame received
got video 2
Frame processed and sent
Frame received
got video 2
Frame processed and sent
Frame received
got video 2
Frame processed and sent
Frame received
got video 2
Frame processed and sent
Frame received
got video 2
Frame processed and sent
Frame received
got video 2
Frame processed and sent
Frame received
got video 2
Frame processed and sent
Frame received
got video 2
Frame processed and sent
Frame received
got video 2
Frame processed and sent
Frame received
got video 2
Frame processed and sent
Frame received
got video 2
Frame processed and sent
Frame received
got video 2
Frame processed and sent
Frame received
got video 2
Frame processed and sent
Frame received
got video 2
Frame processed and sent
Frame received
got video 2
Frame processed and sent
Frame received
got video 2
Frame processed and sent
Frame received
got video 2
Frame processed and 

[2024-05-30 09:03:00] INFO - server.py - connection closed
[2024-05-30 09:08:30] INFO - process.py - t=2024-05-30T09:08:30+0000 lvl=info msg="join connections" obj=join id=da67a60a4862 l=127.0.0.1:8558 r=37.224.52.18:33834
[2024-05-30 09:08:30] INFO - server.py - connection open


...

Client connected
Frame received
got video 2
Frame processed and sent
Frame received
got video 2
Frame processed and sent
Frame received
got video 2
Frame processed and sent
Frame received
got video 2
Frame processed and sent
Frame received
got video 2
Frame processed and sent
Frame received
got video 2
Frame processed and sent
Frame received
got video 2
Frame processed and sent
Frame received
got video 2
Frame processed and sent
Frame received
got video 2
Frame processed and sent
Frame received
got video 2
Frame processed and sent
Frame received
got video 2
Frame processed and sent
Frame received
got video 2
Frame processed and sent
Frame received
got video 2
Frame processed and sent
Frame received
got video 2
Frame processed and sent
Frame received
got video 2
Frame processed and sent
Frame received
got video 2
Frame processed and sent
Frame received
got video 2
Frame processed and sent
Frame received
got video 2
Frame processed and sent
Frame received
got video 2
Frame processed and 

[2024-05-30 09:14:14] INFO - server.py - connection closed
[2024-05-30 09:15:32] INFO - process.py - t=2024-05-30T09:15:32+0000 lvl=info msg="join connections" obj=join id=a715d7cce4c8 l=127.0.0.1:8558 r=37.224.52.18:28304
[2024-05-30 09:15:32] INFO - server.py - connection open


Client connected
Frame received
got video 2
Frame processed and sent
Frame received
got video 2
Frame processed and sent
Frame received
got video 2
Frame processed and sent
Frame received
got video 2
Frame processed and sent
Frame received
got video 2
Frame processed and sent
Frame received
got video 2
Frame processed and sent
Frame received
got video 2
Frame processed and sent
Frame received
got video 2
Frame processed and sent
Frame received
got video 2
Frame processed and sent
Frame received
got video 2
Frame processed and sent
Frame received
got video 2
Frame processed and sent
Frame received
got video 2
Frame processed and sent


[2024-05-30 09:15:45] INFO - server.py - connection closed


Frame received
got video 2
Frame processed and sent
